In [7]:
import logging
import argparse 
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()
%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data-ignored/imdb/'
args.val_fraction = 0.25
args.vocab_size = 2500
args.small_vocab_size = 250
args.epochs = 50
args.batch_size = 32

Path(args.data_folder).mkdir(parents=True, exist_ok=True)

ds, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True, data_dir=args.data_folder)
train_ds_len= tf.data.experimental.cardinality(ds['train']).numpy()
test_ds_len= tf.data.experimental.cardinality(ds['test']).numpy() 
print(train_ds_len)
for d in ds['train'].take(1):
    print(d)
    
# train_dataset = ds['train'].batch(args.batch_size)
train_dataset = ds['train']
val_dataset = ds['test'].take(int(args.val_fraction * (train_ds_len + test_ds_len)))
test_dataset = ds['test'].skip(int(args.val_fraction * (train_ds_len + test_ds_len)))

2021-12-28 17:43:40,704 : INFO : No config specified, defaulting to first: imdb_reviews/plain_text
2021-12-28 17:43:40,707 : INFO : Load dataset info from ./data-ignored/imdb/imdb_reviews/plain_text/1.0.0
2021-12-28 17:43:40,711 : INFO : Reusing dataset imdb_reviews (./data-ignored/imdb/imdb_reviews/plain_text/1.0.0)
2021-12-28 17:43:40,711 : INFO : Constructing tf.data.Dataset imdb_reviews for split None, from ./data-ignored/imdb/imdb_reviews/plain_text/1.0.0


25000
(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


2021-12-28 17:43:41.151842: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [8]:
import functools

@functools.lru_cache(maxsize=10)
def get_encoder(vocab_size=args.vocab_size):
    encoder = TextVectorization(max_tokens=vocab_size)
    encoder.adapt(train_dataset.map(lambda text, label: text))
    return encoder

### Baseline. Bag of words

In [3]:
### Baseline. Bag of words. Preprocessing in dataset creation step.

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def baseline_bag_of_words():
    encoder = get_encoder()
    
    # declaring outputs as an input cause you have to declare tf.variable outside of tf function 
    def build_bag_of_words(tokens, label, outputs=tf.Variable(tf.zeros(args.small_vocab_size))):
        # without it, tf saves the last state of the tensor
        outputs.assign(tf.zeros_like(outputs))
        for i in range(len(tokens)):
            output_idx = tokens[i]
            if output_idx >= tf.constant(args.small_vocab_size, dtype=tf.int64):
                output_idx = tf.constant(1, dtype=tf.int64)
            outputs[output_idx].assign(outputs[output_idx] + 1)
        return outputs, label

    ds_train = train_dataset.map(lambda sent, l: (get_encoder()(sent), l)).map(build_bag_of_words).cache().shuffle(10000).batch(args.batch_size)
    ds_val = val_dataset.map(lambda sent, l: (get_encoder()(sent), l)).map(build_bag_of_words).cache().shuffle(10000).batch(args.batch_size)
    # for d in ds_train.take(10):
    #     print(d)
        
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(250,)))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=keras.optimizers.Nadam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    model.summary()

    
    monitor='val_loss'
    early_stopping = keras.callbacks.EarlyStopping(monitor=monitor, patience=3, mode='auto', restore_best_weights=True, verbose=1)
    reduce_lr_on_plateau = keras.callbacks.ReduceLROnPlateau(monitor=monitor, factor=0.1, patience=2, min_delta=1e-4, mode='auto', verbose=1)
    
    history = model.fit(ds_train, validation_data=ds_val, epochs=args.epochs, callbacks=[early_stopping, reduce_lr_on_plateau])
    
    print('Val_accuracy:', max(history.history['val_accuracy']))
    print('Val_loss:', min(history.history['val_loss']))
    print('Accuracy:', max(history.history['accuracy']))

if True: 
    baseline_bag_of_words()

# val_accuracy: 0.785; val_loss 0.456; accuracy: 0.803

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                16064     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 16,129
Trainable params: 16,129
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


2021-12-20 16:15:14.780774: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 1271 of 10000
2021-12-20 16:15:24.780641: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 2539 of 10000
2021-12-20 16:15:34.780499: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 3782 of 10000
2021-12-20 16:15:44.782971: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 5062 of 10000
2021-12-20 16:15:54.783161: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 6395 of 10000
2021-12-20 16:16:04.782261: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 7649 of 10000
2021-12-20 16:16:14.780100: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling 

779/782 [============================>.] - ETA: 0s - loss: 0.5572 - accuracy: 0.7463 

2021-12-20 16:18:31.684886: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 1342 of 10000
2021-12-20 16:18:41.672330: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 2685 of 10000
2021-12-20 16:18:51.678592: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 3988 of 10000
2021-12-20 16:19:01.669707: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 5270 of 10000
2021-12-20 16:19:11.676965: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 6543 of 10000
2021-12-20 16:19:21.671241: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 7828 of 10000
2021-12-20 16:19:31.676103: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling 

782/782 [==============================] - 294s 275ms/step - loss: 0.5569 - accuracy: 0.7464 - val_loss: 0.4963 - val_accuracy: 0.7651 - lr: 0.0010


2021-12-20 16:19:57.737089: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Epoch 2/50
782/782 [==============================] - 1s 1ms/step - loss: 0.4850 - accuracy: 0.7795 - val_loss: 0.5144 - val_accuracy: 0.7634 - lr: 0.0010
Epoch 3/50
742/782 [===========================>..] - ETA: 0s - loss: 0.4760 - accuracy: 0.7835
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
782/782 [==============================] - 1s 2ms/step - loss: 0.4764 - accuracy: 0.7831 - val_loss: 0.4982 - val_accuracy: 0.7708 - lr: 0.0010
Epoch 4/50
782/782 [==============================] - 1s 1ms/step - loss: 0.4452 - accuracy: 0.7979 - val_loss: 0.4630 - val_accuracy: 0.7837 - lr: 1.0000e-04
Epoch 5/50
782/782 [==============================] - 1s 1ms/step - loss: 0.4418 - accuracy: 0.7992 - val_loss: 0.4616 - val_accuracy: 0.7851 - lr: 1.0000e-04
Epoch 6/50
782/782 [==============================] - 1s 1ms/step - loss: 0.4400 - accuracy: 0.7999 - val_loss: 0.4604 - val_accuracy: 0.7835 - lr: 1.0000e-04
Epoch 7/50
782/782 [===========================

In [5]:
import tensorflow as tf

def stackoverflow_answer():
    class BagOfWords(tf.keras.layers.Layer):
        def __init__(self, vocabulary_size):
            super(BagOfWords, self).__init__()
            self.vocabulary_size = vocabulary_size

        def call(self, inputs):  
            batch_size = tf.shape(inputs)[0]
            outputs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
            for i in range(batch_size):
                string = inputs[i]
                string_length = tf.shape(tf.where(tf.math.not_equal(string, b'')))[0]
                string = string[:string_length]
                string_array = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
                for s in string:
                    string_array = string_array.write(string_array.size(), tf.where(tf.equal(s, self.vocabulary_size), 1.0, 0.0))
                outputs = outputs.write(i, tf.cast(tf.reduce_any(tf.cast(string_array.stack(), dtype=tf.bool), axis=0), dtype=tf.float32))
            return outputs.stack()
        
    labels = [[1], [0], [1], [0]]

    texts  = ['All my cats in a row',
              'When my cat sits down, she looks like a Furby toy!',
              'The cat from the outer space',
              'Sunshine loves to sit like this for some reason.']

    DEFAULT_STRIP_REGEX = r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']'
    tensor_of_strings = tf.constant(texts)
    tensor_of_strings = tf.strings.lower(tensor_of_strings)
    tensor_of_strings = tf.strings.regex_replace(tensor_of_strings, DEFAULT_STRIP_REGEX, "")
    split_strings = tf.strings.split(tensor_of_strings).to_tensor()
    print(split_strings)
    flattened_split_strings = tf.reshape(split_strings, (split_strings.shape[0] * split_strings.shape[1]))
    unique_words, _ = tf.unique(flattened_split_strings)
    unique_words = tf.random.shuffle(unique_words)
    print(unique_words)

    bag_of_words = BagOfWords(vocabulary_size = unique_words)
    train_dataset = tf.data.Dataset.from_tensor_slices((split_strings, labels))
    model = tf.keras.Sequential()
    model.add(bag_of_words)
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss = tf.keras.losses.BinaryCrossentropy())
    model.fit(train_dataset.batch(2), epochs=2)

if False:
    stackoverflow_answer()

tf.Tensor(
[[b'all' b'my' b'cats' b'in' b'a' b'row' b'' b'' b'' b'' b'']
 [b'when' b'my' b'cat' b'sits' b'down' b'she' b'looks' b'like' b'a'
  b'furby' b'toy']
 [b'the' b'cat' b'from' b'the' b'outer' b'space' b'' b'' b'' b'' b'']
 [b'sunshine' b'loves' b'to' b'sit' b'like' b'this' b'for' b'some'
  b'reason' b'' b'']], shape=(4, 11), dtype=string)
tf.Tensor(
[b'sunshine' b'for' b'outer' b'looks' b'down' b'all' b'like' b'this'
 b'she' b'sit' b'toy' b'loves' b'row' b'my' b'the' b'sits' b'some' b'when'
 b'in' b'space' b'furby' b'' b'to' b'cats' b'from' b'reason' b'a' b'cat'], shape=(28,), dtype=string)
Epoch 1/2
2/2 [==============================] - 1s 2ms/step - loss: 0.8469
Epoch 2/2
2/2 [==============================] - 0s 2ms/step - loss: 0.8071


In [ ]:
### Baseline. Bag of words. Preprocessing in a layer.

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def baseline_bag_of_words_layer():
    class BagOfWords(tf.keras.layers.Layer):
        def __init__(self, vocabulary_size):
            super(BagOfWords, self).__init__()
            self.vocabulary_size = vocabulary_size
        def call(self, inputs):  
            batch_size = tf.shape(inputs)[0]
            outputs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
            for i in range(batch_size):
                int_string = inputs[i]
                array_string = tf.TensorArray(dtype=tf.float32, size=self.vocabulary_size)
                array_string.unstack(tf.zeros(self.vocabulary_size))
                for int_word in int_string:
                    idx = int_word
                    idx = tf.cond(idx >= self.vocabulary_size, lambda: 1, lambda: tf.cast(idx, tf.int32))
                    array_string = array_string.write(idx, array_string.read(idx) + 1.0)
                outputs = outputs.write(i, array_string.stack())
            return outputs.stack()
        
    encoder = get_encoder(args.small_vocab_size)
    bag_of_words = BagOfWords(args.small_vocab_size)

    model = keras.models.Sequential()
    model.add(encoder)
    model.add(bag_of_words)
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    # instead of build() on model
    for d in train_dataset.batch(args.batch_size).take(1):
        model(d[0])
    model.compile(optimizer=keras.optimizers.Nadam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    model.summary()
    
    ds_train = train_dataset.shuffle(200000).batch(args.batch_size).prefetch(1)
    ds_val = val_dataset.batch(args.batch_size).prefetch(1)
    monitor='val_loss'
    early_stopping = keras.callbacks.EarlyStopping(monitor=monitor, patience=3, mode='auto', restore_best_weights=True, verbose=1)
    reduce_lr_on_plateau = keras.callbacks.ReduceLROnPlateau(monitor=monitor, factor=0.1, patience=2, min_delta=1e-4, mode='auto', verbose=1)
    
    history = model.fit(ds_train, validation_data=ds_val, epochs=args.epochs, callbacks=[early_stopping, reduce_lr_on_plateau])
    
    print('Val_accuracy:', max(history.history['val_accuracy']))
    print('Val_loss:', min(history.history['val_loss']))
    print('Accuracy:', max(history.history['accuracy']))

if True: 
    baseline_bag_of_words_layer()
    
# Val_accuracy: 0.7857599854469299
# Val_loss: 0.45941051840782166
# Accuracy: 0.7972400188446045

### Rnn with embedding

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

def rnn_with_embedding():
    encoder = get_encoder()
    
    model = keras.models.Sequential()
    model.add(encoder)
    model.add(keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True))
    model.add(keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=keras.optimizers.Nadam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    model.summary()
    
    monitor='val_loss'
    early_stopping = keras.callbacks.EarlyStopping(monitor=monitor, patience=3, mode='auto', restore_best_weights=True, verbose=1)
    reduce_lr_on_plateau = keras.callbacks.ReduceLROnPlateau(monitor=monitor, factor=0.1, patience=2, min_delta=1e-4, mode='auto', verbose=1)
    
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset, callbacks=[early_stopping, reduce_lr_on_plateau])

if False:
    rnn_with_embedding()

# Epoch 3/50
# 782/782 [======] - 314s 401ms/step - loss: 0.2752 - accuracy: 0.8867 - val_loss: 0.3107 - val_accuracy: 0.8667 - lr: 0.0010

### Different embeddings, glove, bert, transformer. 